In [2]:
%load_ext autoreload
%autoreload 2

# import shap
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib as mpl
import matplotlib.pyplot as plt

from tqdm import tqdm
import copy
from IPython.display import clear_output

from util.util_cnn import CNNModel, data_loaders, first_mnist_batch, test, \
                params_to_filename, params_from_filename, load_mnist_v4_models

from util.naming import *
from util.util_lrp import layerwise_forward_pass, compute_relevancies, LRP_global_mat, calc_mats_batch_functional, forward_and_explain
from util.util_tutorial import heatmap
from util.util_pickle import *
from util.util_data_summary import gridlrp_load_results, gridlrp_print_best_and_worst, gridlrp_plot_metric_terrain, gridlrp_plot_metric_terrain_for_tags, \
llrp_plot_training_for_tags

# pan kessel
from explanations_can_be_manipulated.src.nn.enums import ExplainingMethod, LRPRule
from explanations_can_be_manipulated.src.nn.networks import ExplainableNet
from explanations_can_be_manipulated.src.nn.utils import get_expl, plot_overview, clamp, load_image, make_dir

device = 'cpu'
method = ExplainingMethod.lrp

no display found. Using non-interactive Agg backend


In [3]:
%matplotlib inline

In [4]:
# load data and pretrained models
batch_size = 100

data, target = first_mnist_batch(batch_size=batch_size)
background, background_target = first_mnist_batch(batch_size=batch_size, test=False)

background = background.reshape((-1, 1, 28, 28))
test_images = data[:3].reshape((-1, 1, 28, 28))

shap_config = 'shap__background_size-100__batch_size-10__model-cb1-8-8-8_cb2-16-16-16_seed-0'
test_loader_shap = data_loaders(shapley_values_for=('d3', shap_config), shuffle=False, batch_size=batch_size)

model_dict = load_mnist_v4_models()
model_d3 = model_dict[d3_tag]

100%|██████████| 4393/4393 [00:00<00:00, 8368.08it/s] 


## Comparing the Operator norm of every LRP backward step to the 'Operator norm' of the normalization step in the end
The multiplied operator norms are much larger than the factor by which the absolute norm dampens the heatmaps. This is esepcially true for gamma=0.

In [62]:
from functools import partial
from util.util_gamma_rule import calc_vals_batch

l_indices=[0,2,4,6,7,9,11,13,14,15]
for i in l_indices:
    print(i, str(layers[i]).split('(')[0])

n_points = 2
gammas = [0, .5]

mat_funcs = [partial(LRP_global_mat, model=model_d3, l_inp=l_inp, l_out=l_inp+1, delete_unactivated_subnetwork=True) for l_inp in l_indices]
LRP__individual_layer = calc_mats_batch_functional(mat_funcs, gammas, data[:n_points].reshape((n_points, -1)))
LRP__individual_layer.shape, LRP__individual_layer[:, :1, :1]

svals__individual_layer, _ = calc_vals_batch(LRP__individual_layer, num_vals=1, tqdm_for="gamma")

0 Conv2d
2 Conv2d
4 Conv2d
6 MaxPool2d
7 Conv2d
9 Conv2d
11 Conv2d
13 MaxPool2d
14 Flatten
15 Conv2d


  0%|          | 0/10 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
i_point=0

for i, vals in zip(l_indices, svals__individual_layer[:, i_point, :, 0]):
    print(i, str(layers[i]).split('(')[0], '\t', vals.round(1))

0 Conv2d 	 [2.3 2.3]
2 Conv2d 	 [725.8   2.1]
4 Conv2d 	 [1053.6    2.8]
6 MaxPool2d 	 [1. 1.]
7 Conv2d 	 [205.7   2.5]
9 Conv2d 	 [90.4  3. ]
11 Conv2d 	 [258.4   2.4]
13 MaxPool2d 	 [1. 1.]
14 Flatten 	 [1. 1.]
15 Conv2d 	 [3.7 0.8]


In [18]:
from functools import partial
from util.util_gamma_rule import calc_vals_batch

l_indices=[0,2,4,6,7,9,11,13,14,15]
for i in l_indices:
    print(i, str(layers[i]).split('(')[0])

n_points =10
gammas = [0, .5]

mat_funcs = [partial(LRP_global_mat, model=model_d3, delete_unactivated_subnetwork=True)]
LRP__all_layers = calc_mats_batch_functional(mat_funcs, gammas, data[:n_points].reshape((n_points, -1)))
print(LRP__all_layers.shape, LRP__all_layers[:, :1, :1])

svals__all_layers = calc_vals_batch(LRP__all_layers, num_vals=1, tqdm_for="gamma")

0 Conv2d
2 Conv2d
4 Conv2d
6 MaxPool2d
7 Conv2d
9 Conv2d
11 Conv2d
13 MaxPool2d
14 Flatten
15 Conv2d


100%|██████████| 1/1 [00:00<00:00,  5.29it/s]


(1, 10, 2) [[[<139x4 sparse array of type '<class 'numpy.float32'>'
   	with 556 stored elements in COOrdinate format>      ]]]
type(matrices): <class 'numpy.ndarray'>


2it [00:00, 1546.57it/s]
2it [00:00, 2748.56it/s]
2it [00:00, 1152.12it/s]
2it [00:00, 1327.10it/s]
2it [00:00, 1605.17it/s]
2it [00:00, 1755.67it/s]
2it [00:00, 1487.08it/s]
2it [00:00, 1771.24it/s]
2it [00:00, 1199.57it/s]
2it [00:00, 2506.31it/s]


In [31]:
i_point=0
shrinkage_norm = svals__all_layers[0, :, 1, 0] / svals__all_layers[0, :, 0, 0]
print("Smaller by factor of:", shrinkage_norm)

Smaller by factor of: [0.07315576 0.26226059 0.04344419 0.00877301 0.08568073 0.05913895
 0.03357094 0.00762809 0.09217231 0.26065475]


In [73]:
# multiplied operator norms, for two points, for two gammas.
np.product(svals__individual_layer[:, :, :, 0], axis=0).round(2) 

array([[3.08578279e+13, 1.93490000e+02],
       [3.35082642e+13, 1.89740000e+02]])

In [23]:
# ... are much larger than the factor by which the absolute norm dampens the heatmaps. both for gamma=0 ...
model = ExplainableNet(model_d3).eval().to(device)
model.change_lrp_rules(gamma=0, lrp_rule_nl=LRPRule.gamma)
x = data[:10]
heatmap_full, _, _ = get_expl(model, x, method, full=True)
heatmap_full.shape, heatmap_full.abs().sum(axis=(1,2,3))
abs_sum_00 = heatmap_full.abs().sum(axis=(1,2,3)).detach().numpy()
heatmap_full.shape, abs_sum_00

(torch.Size([10, 1, 28, 28]),
 array([122.00882 ,  49.704636,  37.32843 ,  28.613033,  53.84018 ,
        128.03743 ,  94.0966  ,  36.67325 ,  62.04492 , 122.58182 ],
       dtype=float32))

In [22]:
# ... and for gamma=.5 ...
model = ExplainableNet(model_d3).eval().to(device)
model.change_lrp_rules(gamma=0.5, lrp_rule_nl=LRPRule.gamma)
x = data[:10]
heatmap_full, _, _ = get_expl(model, x, method, full=True)
abs_sum_05 = heatmap_full.abs().sum(axis=(1,2,3)).detach().numpy()
heatmap_full.shape, abs_sum_05

(torch.Size([10, 1, 28, 28]),
 array([13.167138 ,  7.9690804, 14.957728 , 16.096722 , 18.535727 ,
        14.438354 , 22.963457 , 13.072809 , 24.076612 ,  9.390237 ],
       dtype=float32))

In [28]:
(shrinkage_normalization := abs_sum_05 / abs_sum_00)

array([0.10791956, 0.16032872, 0.40070605, 0.56256604, 0.34427312,
       0.11276666, 0.2440413 , 0.3564672 , 0.3880513 , 0.07660383],
      dtype=float32)

In [32]:
shrinkage_norm / shrinkage_normalization

array([0.67787303, 1.63576802, 0.10841909, 0.01559464, 0.2488743 ,
       0.52443648, 0.13756255, 0.02139914, 0.23752609, 3.40263335])